In [1]:
import subprocess
import shlex
import os

from subprocess import PIPE
from pathlib import Path

In [2]:
def submit_cuda_spatially_explicit(repeats=10, walltime=1, speciation=0.000001, sample=0.00025, config='()', output='./STDIN'):
    walltime = "{:02d}".format(walltime)
    
    config = f"""
    #PBS -lselect={1}:ncpus={4*1}:mem={24*1}gb:ngpus={1}:gpu_type=P100
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00

    $HOME/necsim-rust/target-cuda/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: CUDA(
            device: {0},
            ptx_jit: true,
            {config.strip()[1:-1]}
        ),

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-base/release/deps/libnecsim_plugins_common.so",
                reporters: [
                    Execution(),
                    Biodiversity(),
                ],
            ),
        ],
    )'
    """
    
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"qsub -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [3]:
for delta_t in [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]:
    submit_cuda_spatially_explicit(
        walltime=5, repeats=10, speciation=0.000001, sample=0.00025,
        config=f"(delta_t:{delta_t})", output=f"delta-t/pbs.{delta_t}",
    )

In [4]:
for event_slice in [100, 316, 1000, 3162, 10000, 31623, 100000, 316228, 1000000]:
    submit_cuda_spatially_explicit(
        walltime=5, repeats=10, speciation=0.000001, sample=0.00025,
        config=f"(parallelism_mode:Monolithic(event_slice:{event_slice}))", output=f"event-slice/pbs.{event_slice}",
    )

In [5]:
for dedup_capacity in [1000, 3162, 10000, 31623, 100000, 316228, 1000000, 3162278, 10000000]:
    submit_cuda_spatially_explicit(
        walltime=5, repeats=10, speciation=0.000001, sample=0.00025,
        config=f"(dedup_cache:Absolute(capacity:{dedup_capacity}))",
        output=f"dedup-capacity/pbs.{dedup_capacity}",
    )

In [3]:
for block_size in [4, 16, 32, 64, 128, 256, 512, 1024]:
    for grid_size in [4, 16, 64, 256, 1024]:
        submit_cuda_spatially_explicit(
            walltime=5, repeats=10, speciation=0.000001, sample=0.00025,
            config=f"(block_size:{block_size},grid_size:{grid_size})",
            output=f"block-grid/pbs.{block_size}.{grid_size}",
        )

In [3]:
for (speciation, sample) in [
    (0.000001, 0.00025), (1.0, 0.00025), (0.01, 0.000025), (0.0001, 0.0000025), (0.000001, 0.00000025), 
]:
    for step_slice in [10, 50, 100, 150, 200, 250, 300, 350]:
        submit_cuda_spatially_explicit(
            walltime=5, repeats=10, speciation=speciation, sample=sample,
            config=f"(step_slice:{step_slice})",
            output=f"step-slice/pbs.{speciation}.{sample}.{step_slice}",
        )